'''
This program is a version of the futureperfect_datamangaement_createparagraphs 
but it only looks for the word "future" explicitly
#Date: Dec 3 2019
#Update: Mar 25, 2019
#update: July 7, 2020
#Author: Carly Knight
'''

### Import packages

In [1]:
import os
from zipfile import ZipFile
import xml.etree.ElementTree as ET
from xml import etree
import csv
import re
import dateutil.parser as parser
from enchant.tokenize import get_tokenizer
import enchant
import pandas as pd
from itertools import chain
import random

## get liwc list


In [2]:
#add/remove
futurelist_word= ["Future", "future"] 

#financial statement list
financialstatement = ["Notes to Consolidated Financial Statements", "Notes to Financial Statements"]

#paragrapsh to exclude list
phrasetoexclude = financialstatement + ['financial accounting', "the following table", "transfer agent", "annual meeting of shareholders", "annual meeting of stockholders", "next annual meeting"]

notwordlist = ["futures", "Futures"]

print(futurelist_word)
#Other potential options:
#Risk, Money, Quant, Verb, Tentat, Certain, Time

['Future', 'future']


# Prep + helper functions

### Input and Output Data Location

In [18]:
directory = "/Users/carlyknight/Documents/Data/Annual Report/XML/"
results= "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"
results_paragraphs = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts_paragraphs_futureonly/"
test_files= "/Users/carlyknight/Dropbox/PROJECTS/ConceptionsofRisk/data/keyword_test_files/"
test_paragraphs = "/Users/carlyknight/Dropbox/PROJECTS/Corporate_Futures_for_Hesu/paragraphs/"

### Help function: spell checker

In [3]:
#function to calculate the percent
def calculate_percent_correct(text):
	true =0
	total = 0.0
	for word in tknzr(text):
		word1 =word[0]
		total +=1
		if d2.check(word1):
			true +=1
	if total !=0:
		percent = true*1.0/total
	else:
		percent =0
	return percent

'''INITIATE SPELLCHECKER'''
d2 = enchant.DictWithPWL("en_US")
tknzr = get_tokenizer("en_US")

### Helper function: clean text

In [4]:
def get_fulltext(root):
	full = root.find('FullText')
	if full is None:
		fulltext = ""
	else:
		fulltext=full.text
	return fulltext

In [5]:
def clean_punction(fulltext):
    #remove various kinds of superfluous punction
    fulltext = re.sub(r'[;:$-*><=|^·\,\\-]','',fulltext)
    fulltext.replace(u'No.', u'No')
    fulltext.replace("\\", "") 
    #replace series of periods with one period
    fulltext = re.sub(r'(\.\s*\.\s*)+', '', fulltext)
    return fulltext

### Regex Compile

In [7]:
pattern_word = r"(?<=\W)(%s)\b" % "|".join(futurelist_word)
regex_word = re.compile(pattern_word)

pattern_finance = r"(?<=\W)(%s)" % "|".join(financialstatement)
regex_financestatement = re.compile(pattern_finance)

#note: we don't care about case sensitivity for the removals
pattern_remove = r"(?<=\W)(%s)" % "|".join(phrasetoexclude)
regex_remove = re.compile(pattern_remove, re.IGNORECASE)

### Helper fuction: Wordsearch functions

In [8]:
#identify paragraphs
def paragraph_split(fulltext):
    splits= re.split('(\.{3,}[-_\s]*\.{2,}[-_\s]*)|(\_{3,})|(\-{3,})|(\s{3,}[0-9]{1,}\s{3,})|(\s{5,}[A-Z\s]+\s{5,})', fulltext)
    splits=[split for split in splits if split is not None if split.strip() is not '']
    return splits

#combine paragraphs if they are too short
def paragraph_combine(si, length):
    si = iter(si)
    while True:
        try:
            current = next(si)
        except StopIteration:
            return
        try:
            while len(current) < length:
                current += ' ' + next(si)
        except StopIteration:
            return
        try:
            yield current
        except StopIteration:
            return
        

In [9]:
#identify only paragraphs with a given data quality
def select_good_paragraphs(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #remove paragraphs that begin with "Note"
        if paragraph.split()[0] == "Note" or paragraph.split()[0] == "NOTE":
            continue
        # keep only paragraphs that aren't just numbers 
        nums=sum(c.isdigit() for c in paragraph)
        percent_digits = (nums*1.0)/len(paragraph)
        #keep sections with spell correctness above .85
        spellcorrect= calculate_percent_correct(paragraph)
        #remove sections with list of all capitalized words (most likely to be names)
        percent_uppers = (1.0*sum(1 for c in paragraph if c.isupper()))/len(paragraph)
        if percent_digits <.12 and percent_uppers <.5 and spellcorrect >.85 :
            selected_paragraphs.append(paragraph)
    return selected_paragraphs

In [10]:
#identify future paragraphs
def paragraph_wordsearch(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #search paragraph for wordlist
        num_matches=len(regex_word.findall(paragraph))
        if num_matches > 0: #if there is a match
            #chain the word and stem iterators:
            chain_iter = chain(regex_word.finditer(paragraph))
            #iterate through chain
            for match in chain_iter:
                #make sure only match is not in not-match list:
                if [m for m in notwordlist if m in paragraph[match.start()-10: match.end()+10]]:
                    continue
                #else:
                 #replace with asterisks
                paragraph =paragraph.replace(match.group(), "***" + match.group())
            #substitute due to loop problems
            paragraph = re.sub(r'\*+', "***", paragraph)
            #if there are asterisks, then append
            if "***" in paragraph:
                selected_paragraphs.append(paragraph)
    return selected_paragraphs


In [11]:
#remove paragraphs contain content we know we don't want
def paragraph_remove(paragraphs):
    selected_paragraphs = []
    for paragraph in paragraphs:
        #search for what to remove
        if len(regex_remove.findall(paragraph))>0: #if there is something to remove
            #print("FOUND PARAGRAPH TO REMOVE")
            continue
        elif "NOTE" in paragraph:
            #print("WORD 'NOTE' FOUND")
            continue
        else:
            selected_paragraphs.append(paragraph)
    return selected_paragraphs


In [12]:
#identify future paragraphs for examples
def paragraph_example_wordsearch(selected_text):
    paragraphs = selected_text.split("####")
    for paragraph in paragraphs: #iterate through paragraph
        for i,item in enumerate(example_list): #for each paragraph, test for each item
            if re.search(r"(?<=\*\*\*)" + item + r"\b", paragraph):
                example_container_lists[i].append(paragraph)

In [13]:
#find financial statement and remove text after
def find_financialstatement(fulltext):
    matches = regex_financestatement.finditer(fulltext)
    text_length = len(fulltext)
    if matches: 
        for m in matches:
            location = m.start()
            finance_location = location/text_length
            #print(m.start(), m.group(), finance_location)
            #if finance location in back fourth of document, remove everything after that location
            if finance_location >= 2/3:
                fulltext = fulltext[:location]
                return(fulltext)
                break
    return(fulltext)

### Helper function: get metadata

In [14]:
def get_metadata(root):
    date= root.find('AlphaPubDate').text
    datenum = root.find('NumericPubDate').text
    year = parser.parse(date).year
    url = root.find('URLDocView').text
    child = root.find('Terms')
    #create empty containers for terms
    company = ''
    naics=''
    code = ''
    classic = ''
    #iterate through terms
    for alltags in child.findall('.//'):
        if alltags.tag == "CompanyName":
            company = alltags.text
        elif alltags.tag == "CompanyNAIC":
            naics = alltags.text
        elif alltags.tag == "ClassCode":
            code = alltags.text
        elif alltags.tag == "ClassExpansion":
            classic = alltags.text
    # put the flexterms into a dictionary
    dict = {}
    for elem in root.iter('FlexTerm'):
        dict[elem.find('FlexTermName').text] = elem.find('FlexTermValue').text
    # create terms from dictionary
    ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns = [dict.get(k) for k in
                                                                            ['AnrClassCode1IdxLit',
                                                                             'AnrCashDispIdxNum',
                                                                             'AnrCashSrchIdxNum',
                                                                             'AnrAssetDispIdxNum',
                                                                             'AnrAssetSrchIdxNum',
                                                                             'AnrRevDispIdxNum',
                                                                             'AnrRevSrchIdxNum',
                                                                             'AnrEarnDispIdxNum',
                                                                             'AnrEarnSrchIdxNum']]
    # if variables above are NONE, recode as empty string
    ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns = [k or "" for k in
                                                                            [ancode, ancashd, ancashs, anasd,
                                                                             anass, anrevd, anrevs, anearnd,
                                                                             anearns]]
    #create list of alternative company names
    altnames =[]
    for elem in root.iter('FlexTerm'):
        if elem.find('FlexTermName').text == "AnrRelCoNameIdxLit":
            altnames.append(elem.find('FlexTermValue').text)
    relnames='; '.join(altnames)
    return date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames
    

# Main loop

### create list for random selection to check

In [15]:
random_selection = []

### create list of zip-files

In [19]:
zippaths=[]
for zipped in os.listdir(directory):
	if not zipped.endswith('.zip'):continue
	zippaths.append(os.path.join(directory, zipped))


### open csv for metadata- paragraph

In [20]:
csvfile = open(results_paragraphs + 'metadata_futureperfect_paragraphs_futureonly.csv', 'w')
fieldnames = ['Location', 'Filename', "Paragraph", 'RecordTitle', 'Date', 'Date1', 'Year', 'URL', 'CompanyName', 'NAIC', 'ClassCode', 'Classification', 'AnrAssetDispIdxNum', 'numwordstotal', "numwordsparagraph", 'num_paragraphs_postspellcheck']
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
writer.writeheader()
            

# TEST

In [21]:
# zip1 = [zippaths[4]]
# for filename in zip1:
#     with ZipFile(filename, "r") as zf:
#         for name in zf.namelist():
#             print("FILE :", name)
#             data= zf.read(name)
#             root = ET.fromstring(data)
#             date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames= get_metadata(root)
#             #get fulltext
#             fulltext= get_fulltext(root)
#             #clean fulltext
#             fulltext=clean_punction(fulltext)
#             #find and remove financial statements later in document
#             fulltext=find_financialstatement(fulltext)
#             #identify paragraphs
#             paragraphs1 = paragraph_split(fulltext)
#             paragraphs2 = list(paragraph_combine(paragraphs1, 200))
#             #wordsearch paragraphs
#             paragraphs3= paragraph_wordsearch(paragraphs2)
#             n_paragraphs_pre= len(paragraphs3)
#             #select only quality paragraphs
#             paragraphs4 = select_good_paragraphs(paragraphs3)
#             #remove bad phrases
#             paragraphs5 = paragraph_remove(paragraphs4)
#             if paragraphs5 is None or len(paragraphs5)==0:
#                 print(str(i), name, "No results returned")
#                 continue
#             #combine paragraphs again for STM analysis (into 500 words)
#             paragraphs6 = paragraph_combine(paragraphs5, 500)
           

# Loop

In [22]:
i=1
for filename in zippaths:
    with ZipFile(filename, "r") as zf:
        for name in zf.namelist():
            data= zf.read(name)
            root = ET.fromstring(data)
            date, datenum, year, url, company, naics, code, classic, ancode, ancashd, ancashs, anasd, anass, anrevd, anrevs, anearnd, anearns, relnames= get_metadata(root)
            #get fulltext
            fulltext= get_fulltext(root)
            #clean fulltext
            fulltext=clean_punction(fulltext)
            #find and remove financial statements later in document
            fulltext=find_financialstatement(fulltext)
            #identify paragraphs
            paragraphs1 = paragraph_split(fulltext)
            paragraphs2 = list(paragraph_combine(paragraphs1, 200))
            #wordsearch paragraphs
            paragraphs3= paragraph_wordsearch(paragraphs2)
            #select only quality paragraphs
            paragraphs4 = select_good_paragraphs(paragraphs3)
            #remove bad phrases
            paragraphs5 = paragraph_remove(paragraphs4)
            #combine paragraphs again for STM analysis (into 500 words)
            paragraphs6 = list(paragraph_combine(paragraphs5, 500))
            n_paragraphs_post= len(paragraphs6)
            if paragraphs6 is None or len(paragraphs6)==0:
                print(str(i), name, "No results returned")
                continue
            #select a random paragraph for Barbara analysis in post 1900 period
            if year >1990:
                selection = random.choice(paragraphs6)
                random_selection.append([name, selection])
            #paragraph_example_wordsearch(selected_text)
            #join text
            selected_text= "####".join(paragraphs6)
            #calculate number of words
            numwordstotal = len(selected_text.split())
            #write out full document
            with open(results + re.sub(".xml", ".txt", name), "w") as text_file:
                _=text_file.write(selected_text)
            #iterate through paragraphs and write file
            for p, paragraph in enumerate(paragraphs6):
                #length of paragraph
                numwordspara = len(paragraph)
                #put results in a dict
                resultsdict= {'Filename': name, "Paragraph": p, 'RecordTitle': id, 'Date': date, "Date1": datenum, 'Year': year, 'URL': url, 'CompanyName': company.encode('utf-8'), 'NAIC': naics, 'ClassCode': code, 'Classification': classic.encode('utf-8'), 'AnrAssetDispIdxNum': anasd, 'numwordstotal': numwordstotal, 'numwordsparagraph': numwordspara, 'num_paragraphs_postspellcheck': n_paragraphs_post}
                #write to metadata file
                writer.writerow(resultsdict)
                #write out selected text
                filenm = re.sub(".xml", "", name)
                pname = filenm + "_P" + str(p) + ".txt"
                with open(results_paragraphs + pname, "w") as text_file2:
                    _=text_file2.write(paragraph)
            print(str(i), name, "Written")
            i+=1
            
            

1 88201676.xml Written
2 88179860.xml No results returned
2 88198679.xml Written
3 88190194.xml No results returned
3 88178029.xml No results returned
3 88198719.xml Written
4 88177949.xml Written
5 88180180.xml Written
6 88205316.xml Written
7 88180349.xml Written
8 88203156.xml Written
9 88179900.xml Written
10 88179100.xml Written
11 88178669.xml No results returned
11 88175669.xml No results returned
11 88197156.xml Written
12 88177269.xml No results returned
12 88201519.xml Written
13 88193154.xml Written
14 88200119.xml Written
15 88179620.xml No results returned
15 88176989.xml No results returned
15 88178229.xml No results returned
15 88191354.xml No results returned
15 88179420.xml Written
16 88197999.xml Written
17 88176869.xml Written
18 88189354.xml Written
19 88175429.xml No results returned
19 88204516.xml Written
20 88178909.xml No results returned
20 88173700.xml No results returned
20 88180420.xml No results returned
20 88177220.xml No results returned
20 88176909.xml 

167 88192558.xml No results returned
167 88212361.xml No results returned
167 88176994.xml No results returned
167 88206364.xml No results returned
167 88204964.xml Written
168 88184790.xml No results returned
168 88202946.xml No results returned
168 88186712.xml No results returned
168 88204786.xml Written
169 88173017.xml Written
170 88197720.xml Written
171 88191629.xml Written
172 88175297.xml Written
173 88174016.xml No results returned
173 88173975.xml No results returned
173 88182027.xml No results returned
173 88182507.xml Written
174 88174735.xml No results returned
174 88203782.xml No results returned
174 88200300.xml Written
175 88177348.xml No results returned
175 88191874.xml Written
176 88179508.xml No results returned
176 88206342.xml Written
177 88173610.xml Written
178 88206625.xml No results returned
178 88192114.xml Written
179 88186342.xml No results returned
179 88173417.xml Written
180 88195354.xml Written
181 88173170.xml No results returned
181 88195960.xml Writ

314 88177171.xml Written
315 88187216.xml No results returned
315 88186097.xml Written
316 88190976.xml Written
317 88179298.xml No results returned
317 88191059.xml Written
318 88190056.xml Written
319 88189419.xml Written
320 88185337.xml Written
321 88179658.xml No results returned
321 88185017.xml No results returned
321 88181417.xml No results returned
321 88192016.xml No results returned
321 88191816.xml Written
322 88189859.xml No results returned
322 88187296.xml Written
323 88178017.xml No results returned
323 88188256.xml Written
324 88195216.xml No results returned
324 88177131.xml No results returned
324 88182818.xml Written
325 88190496.xml No results returned
325 88178098.xml No results returned
325 88179818.xml Written
326 88180938.xml No results returned
326 88178131.xml No results returned
326 88178251.xml No results returned
326 88177257.xml No results returned
326 88179138.xml Written
327 88176811.xml No results returned
327 88184937.xml Written
328 88194176.xml Writ

478 88189123.xml Written
479 88183642.xml Written
480 88196234.xml No results returned
480 88197070.xml No results returned
480 88183403.xml No results returned
480 88188927.xml No results returned
480 88194470.xml No results returned
480 88193634.xml No results returned
480 88205036.xml Written
481 88186807.xml No results returned
481 88173374.xml Written
482 88187323.xml No results returned
482 88195475.xml Written
483 88194110.xml No results returned
483 88175574.xml No results returned
483 88189795.xml No results returned
483 88194235.xml Written
484 88194430.xml Written
485 88176290.xml No results returned
485 88190994.xml No results returned
485 88174010.xml No results returned
485 88188714.xml Written
486 88196751.xml Written
487 88194475.xml Written
488 88197591.xml Written
489 88184883.xml No results returned
489 88195351.xml Written
490 88184122.xml Written
491 88205516.xml Written
492 88193790.xml No results returned
492 88203436.xml Written
493 88184643.xml Written
494 8817

667 88225696.xml Written
668 88195419.xml Written
669 88194983.xml Written
670 88186210.xml Written
671 88205282.xml Written
672 88195639.xml Written
673 88188344.xml No results returned
673 88179212.xml No results returned
673 88191840.xml Written
674 88192347.xml Written
675 88190298.xml No results returned
675 88202876.xml Written
676 88192005.xml No results returned
676 88200563.xml Written
677 88193659.xml Written
678 88207668.xml Written
679 88189298.xml Written
680 88186022.xml No results returned
680 88186389.xml No results returned
680 88189667.xml Written
681 88196176.xml No results returned
681 88224347.xml Written
682 88176372.xml No results returned
682 88194943.xml Written
683 88208107.xml Written
684 88200319.xml Written
685 88186698.xml No results returned
685 88200499.xml Written
686 88199543.xml Written
687 88198259.xml Written
688 88186229.xml Written
689 88195776.xml Written
690 88185463.xml No results returned
690 88198623.xml Written
691 88196383.xml Written
692 8

...

28190 88201775.xml Written
28191 88178265.xml No results returned
28191 88177201.xml No results returned
28191 88208389.xml Written
28192 88209349.xml Written
28193 88205109.xml Written
28194 88177041.xml No results returned
28194 88199867.xml Written
28195 88194575.xml No results returned
28195 88196770.xml Written
28196 88173241.xml No results returned
28196 88194095.xml No results returned
28196 88200690.xml Written
28197 88201250.xml Written
28198 88177801.xml Written
28199 88191606.xml No results returned
28199 88194046.xml No results returned
28199 88199010.xml Written
28200 88197135.xml Written
28201 88195810.xml No results returned
28201 88191886.xml No results returned
28201 88227203.xml Written
28202 88211720.xml Written
28203 88230529.xml Written
28204 88230843.xml Written
28205 88223123.xml Written
28206 88214120.xml Written
28207 88224600.xml Written
28208 88229443.xml No results returned
28208 88222000.xml Written
28209 88213240.xml Written
28210 88228683.xml Written
2821

28387 88191065.xml Written
28388 88189665.xml No results returned
28388 88198680.xml Written
28389 88202380.xml Written
28390 88195260.xml Written
28391 88196840.xml Written
28392 88200060.xml No results returned
28392 88197360.xml Written
28393 88193045.xml Written
28394 88199300.xml Written
28395 88199085.xml Written
28396 88191222.xml No results returned
28396 88191585.xml No results returned
28396 88196080.xml Written
28397 88195925.xml Written
28398 88198625.xml Written
28399 88192820.xml No results returned
28399 88197020.xml No results returned
28399 88187542.xml No results returned
28399 88188142.xml Written
28400 88201285.xml Written
28401 88196505.xml Written
28402 88191700.xml Written
28403 88194565.xml Written
28404 88201225.xml Written
28405 88201905.xml Written
28406 88223929.xml Written
28407 88201747.xml Written
28408 88230206.xml Written
28409 88212683.xml Written
28410 88229988.xml Written
28411 88227120.xml Written
28412 88203669.xml No results returned
28412 8823148

28602 88221280.xml Written
28603 88224769.xml Written
28604 88231563.xml Written
28605 88176004.xml No results returned
28605 88220680.xml Written
28606 88206560.xml Written
28607 88178644.xml Written
28608 88231882.xml Written
28609 88218080.xml Written
28610 88215800.xml Written
28611 88229449.xml Written
28612 88179049.xml No results returned
28612 88176044.xml No results returned
28612 88177969.xml No results returned
28612 88229605.xml Written
28613 88175480.xml No results returned
28613 88179969.xml Written
28614 88179200.xml No results returned
28614 88229089.xml Written
28615 88229845.xml Written
28616 88226400.xml Written
28617 88175369.xml No results returned
28617 88176720.xml No results returned
28617 88225323.xml Written
28618 88175604.xml No results returned
28618 88179089.xml No results returned
28618 88229565.xml Written
28619 88175684.xml Written
28620 88179880.xml No results returned
28620 88224249.xml Written
28621 88229169.xml Written
28622 88232282.xml No results r

28775 88195026.xml No results returned
28775 88176503.xml No results returned
28775 88183925.xml No results returned
28775 88192945.xml No results returned
28775 88201416.xml Written
28776 88186728.xml No results returned
28776 88202219.xml Written
28777 88182343.xml Written
28778 88188840.xml Written
28779 88194865.xml Written
28780 88183285.xml No results returned
28780 88196129.xml Written
28781 88180621.xml Written
28782 88229644.xml Written
28783 88218466.xml Written
28784 88204926.xml No results returned
28784 88174983.xml No results returned
28784 88191014.xml Written
28785 88208386.xml Written
28786 88178741.xml No results returned
28786 88204486.xml Written
28787 88190324.xml Written
28788 88199726.xml Written
28789 88192324.xml Written
28790 88199806.xml No results returned
28790 88221906.xml Written
28791 88198891.xml Written
28792 88189604.xml No results returned
28792 88193448.xml Written
28793 88197614.xml Written
28794 88197531.xml No results returned
28794 88189246.xml 

28981 88229201.xml Written
28982 88230281.xml Written
28983 88221508.xml Written
28984 88212388.xml Written
28985 88225084.xml Written
28986 88213164.xml Written
28987 88218884.xml Written
28988 88185085.xml No results returned
28988 88196347.xml No results returned
28988 88216844.xml Written
28989 88200188.xml Written
28990 88204144.xml Written
28991 88192683.xml No results returned
28991 88199563.xml Written
28992 88193403.xml Written
28993 88201763.xml Written
28994 88194363.xml No results returned
28994 88197683.xml Written
28995 88198003.xml Written
28996 88206064.xml Written
28997 88206304.xml Written
28998 88207367.xml Written
28999 88203628.xml Written
29000 88184001.xml Written
29001 88181441.xml No results returned
29001 88207789.xml Written
29002 88231308.xml Written
29003 88194415.xml Written
29004 88194655.xml No results returned
29004 88199347.xml Written
29005 88193846.xml Written
29006 88203787.xml Written
29007 88231507.xml Written
29008 88206909.xml Written
29009 8820

## Create text file with examples

In [192]:
#for i,item in enumerate(example_list):
#    with open(test_files +  item.replace(" ", "") + '2.txt', 'w') as f:
#        f.write('%s\n\n' % example_container_lists[i])

## Create selection of 1000 random paragrahs

In [1]:
#how many were selected?
len(random_selection)

NameError: name 'random_selection' is not defined

In [147]:
#select 1000 random paragraphs 
random_sample = random.choices(random_selection, k=1000)
len(random_sample)

1000

In [148]:
#put these in a csv
with open(test_paragraphs + "random_sample_7JULY.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(random_sample)